In [15]:
# 1.라이브러리 불러오기
import pandas as pd
import numpy as np

import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense

In [16]:
# 2. 데이터 불러오기
train_df = pd.read_csv(r'C:\Users\kwonk\Downloads\개인 프로젝트\juno1412-1\DL\모두의 딥러닝\ml_10_medicalalert_train.csv')
test_df = pd.read_csv(r'C:\Users\kwonk\Downloads\개인 프로젝트\juno1412-1\DL\모두의 딥러닝\ml_10_medicalalert_test.csv')

# 데이터 체크
train_df.head()
test_df.head()

,id,patientid,timestamp,gender,age,di,copd,chf,ht,afib,w,pw,bps,bpd,spo2,hr,gl
0,1,986452681195952909,2015-07-28 16:56:36,F,75,1.0,0,0.0,1,1,62.595696,62.595696,108.0,75.0,97.0,85.0,208.0
1,2,279227943463908135,2015-07-19 18:04:45,F,60,1.0,1,0.0,1,1,104.779752,104.280801,176.0,99.0,96.0,81.0,NaN
2,3,190474597341608800,2015-08-11 0:10:17,M,76,0.0,0,0.0,0,0,110.676448,107.954896,139.0,85.0,NaN,91.0,NaN
3,4,1059985795439793914,2015-07-05 12:42:54,M,79,1.0,0,1.0,1,0,103.917930,102.511792,114.0,46.0,92.0,77.0,84.0
4,5,533141523892671381,2015-07-02 16:05:42,M,80,1.0,0,0.0,1,1,55.519661,55.701098,111.0,57.0,95.0,71.0,124.0


In [17]:
# 3. 데이터 전처리
# 3-1. train_set 전처리
# string data labeling
train_df = train_df.replace({'gender': {'M':0,'F':1}, 'alert' : {'Yes':0, 'No':1}}).fillna(0.0)

# patientid, timestamp 제거 모델 예측에 필요없음
train_df2 = train_df.drop(columns = ['patientid', 'timestamp'], axis=1)

# object data type -> float변경
# model input
train_df2 = train_df2.apply(pd.to_numeric, errors='coerce').fillna(0.0)
train_df2 = train_df2.values



In [18]:
# 3-2. test_set 젙처리
# string data labeling
test_df = test_df.replace({'gender': {'M':0,'F':1}, 'alert' : {'Yes':0, 'No':1}}).fillna(0.0)

# patientid, timestamp 제거 모델 예측에 필요없음
test_df2 = test_df.drop(columns = ['patientid', 'timestamp'], axis=1)

# object data type -> float변경
# model input
test_df2 = test_df2.apply(pd.to_numeric, errors='coerce').fillna(0.0)
# id를 인덱스로 설정
test_df2 = test_df2.set_index('id')
test_df2 = test_df2.values

In [19]:
# 3-3. input-output data 전처리
x = train_df2[:,0:14]

# 3-2. 데이터 라벨링
y = train_df2[:,14]
e = LabelEncoder() # 각 class 문자열 -> 숫자로 변환
e.fit(y)
y_label = e.transform(y)


In [20]:
# 모델
# 4. 모델
model = Sequential()
model.add(Dense(32, input_dim=14, activation ='relu'))
model.add(Dense(32, activation ='relu'))
model.add(Dense(16, activation ='relu'))
model.add(Dense(1, activation ='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics=['acc'])

# adam에서 최고성능 98.88% 정확도 수렴 -> epoch 10
model.fit(x, y_label, epochs=10, batch_size =50, validation_split=0.2)

Epoch 1/10
400/400 [==============================] - 3s 4ms/step - loss: 0.6224 - acc: 0.9589 - val_loss: 0.0767 - val_acc: 0.9872
Epoch 2/10
400/400 [==============================] - 1s 4ms/step - loss: 0.0697 - acc: 0.9869 - val_loss: 0.0736 - val_acc: 0.9882
Epoch 3/10
400/400 [==============================] - 1s 3ms/step - loss: 0.0641 - acc: 0.9877 - val_loss: 0.0633 - val_acc: 0.9880
Epoch 4/10
400/400 [==============================] - 1s 4ms/step - loss: 0.0640 - acc: 0.9880 - val_loss: 0.0616 - val_acc: 0.9874
Epoch 5/10
400/400 [==============================] - 1s 4ms/step - loss: 0.0587 - acc: 0.9885 - val_loss: 0.0614 - val_acc: 0.9880
Epoch 6/10
400/400 [==============================] - 1s 4ms/step - loss: 0.0581 - acc: 0.9883 - val_loss: 0.0607 - val_acc: 0.9874
Epoch 7/10
400/400 [==============================] - 2s 4ms/step - loss: 0.0576 - acc: 0.9884 - val_loss: 0.0641 - val_acc: 0.9876
Epoch 8/10
400/400 [==============================] - 1s 4ms/step - loss: 0.

In [26]:
# test_data 예측값 도출
test_df['alert'] = model.predict(test_df2)
test_df['alert'].to_csv(r'C:\Users\kwonk\Downloads\개인 프로젝트\juno1412-1\DL\모두의 딥러닝\test_result.csv', index=False)

print('done')

223/223 [==============================] - 1s 3ms/step
done
